Source of scraping script: https://www.kaggle.com/code/michaelbaumgartner/download-nasdaq-historical-data/edit

In [1]:
offset = 0
# limit = 3000
period = '10y' # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max

In [2]:
import pandas as pd

data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
data_clean = data_clean[data_clean['ETF']=='N']
symbols = data_clean['NASDAQ Symbol'].tolist()
print('total number of symbols traded = {}'.format(len(symbols)))

total number of symbols traded = 9273


In [11]:
a = pd.read_csv('../merged_posts1.csv')
a

,title,score,num_comments,body,created,stock
0,AAPL vs TSLA apes near ATH's,22,9,NaN,1.649159e+09,AAPL
1,$TSLA prediction using Wyckoff Result vs Effor...,28,34,"First off, I'm not a financial advisor, I am a...",1.648973e+09,AAPL
2,"I SPY, AAPL, GME, and TSLA TA - 4/4 Scalpers D...",39,10,"# Economic Calendar - April 4, 2022\n\nhttps:/...",1.648909e+09,AAPL
3,SPY & AAPL Gains this past week,48,14,NaN,1.648855e+09,AAPL
4,Eating an AAPL in My TSLA Outside GME - Scalpe...,152,32,>Don't be a square when you can be a rectangle...,1.648684e+09,AAPL
...,...,...,...,...,...,...
270,[SERIOUS DD] on **TSMC**,73,34,"Ok guys, so my weekly 2-day WSB ban has expire...",1.536718e+09,MFG
271,"Upcoming Catalysts – Week Ending Friday, Octob...",204,60,Hello and happy Thursday night or Friday morni...,1.475196e+09,MFG
272,NVDA supply chain / mfg issues?,3,7,Hear me out here: \nSo I'm slightly intereste...,1.470760e+09,MFG
273,FSLR DD and speculation,2,2,1. First solar has a conservative and vertical...,1.460428e+09,MFG


In [9]:
! pip3 install yfinance > /dev/null 2>&1
! mkdir hist

In [4]:
import yfinance as yf
import os, contextlib

In [5]:
%%time

limit = len(symbols)
end = min(offset + limit, len(symbols))
is_valid = [False] * len(symbols)
# force silencing of verbose API
with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull):
        for i in range(offset, end):
            s = symbols[i]
            data = yf.download(s, period=period)
            if len(data.index) == 0:
                continue
        
            is_valid[i] = True
            data.to_csv('hist/{}.csv'.format(s))

print('Total number of valid symbols downloaded = {}'.format(sum(is_valid)))

In [ ]:
valid_data = data_clean[is_valid]
valid_data.to_csv('symbols_valid_meta.csv', index=False)

In [ ]:
!mkdir stocks
!mkdir etfs

In [ ]:
etfs = valid_data[valid_data['ETF'] == 'Y']['NASDAQ Symbol'].tolist()
stocks = valid_data[valid_data['ETF'] == 'N']['NASDAQ Symbol'].tolist()

In [ ]:
import shutil
from os.path import isfile, join

def move_symbols(symbols, dest):
    for s in symbols:
        filename = '{}.csv'.format(s)
        shutil.move(join('hist', filename), join(dest, filename))
        
move_symbols(etfs, "etfs")
move_symbols(stocks, "stocks")

In [ ]:
! rmdir hist